#  RAG Example (small but complete)

For more information, see https://python.langchain.com/docs/tutorials/rag/ 

- (saved) Vector DB (Chroma)
- Ollama

## Specify embedding model and vector store

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings()
database_loc = ("./chroma_db_test1")

vectorstore = Chroma(persist_directory=database_loc,
      embedding_function=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 8})

## Specify the LLM

We are going to use Ollama to keep it simple

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = OllamaLLM(model='llama3.1:8b') 

## Create a simple prompt template

This can be handled in many ways and can be very simple to complex.

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """You are an AI counselor. Answer questions using the provided information. 
Make sure to cover as much information as possible.
{context}
Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)

## Create a chain of steps

For more information, see https://python.langchain.com/docs/tutorials/rag/ 

In [ ]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vectorstore.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}

## Connect them and save them as a Langgraph

This is quite new. Chain creation was done thorugh piping until recently.

In [ ]:
from langgraph.graph import START, StateGraph

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

## Now comes the fun part!

In [ ]:
response = graph.invoke({"question": "Are there Japanese classes offered at Lafayette?"})

print(f'Answer: {response["answer"]}\n\n')
print("*" *80)      
print(f'Context: {response["context"]}\n\n')

In [ ]:
response = graph.invoke({"question": "Can I learn Chinese at Lafayette High School?"})

print(f'Answer: {response["answer"]}\n\n')
print("*" *80)      
print(f'Context: {response["context"]}\n\n')

In [ ]:
response = graph.invoke({"question": "I am interested in designing houses. What can I take at Lafayette?"})

print(f'Answer: {response["answer"]}\n\n')
print("*" *80)      
print(f'Context: {response["context"]}\n\n')